In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans 

import warnings
warnings.filterwarnings("ignore")

# Cleaning and merging all listings datasets

In [ ]:
#Creating a function to edit each df and creating a list of them to concatenate
listings_Bronx = []
listings_Brooklyn = []
listings_Manhattan = []
listings_Queens = []
listings_StatenIsland = []

#Getting the common columns
column_list = ["id","neighbourhood_group_cleansed","neighbourhood_cleansed","latitude","longitude",
                   "room_type","property_type","accommodates","bedrooms","beds","price","amenities"]

property_list = ["Apartment","House","Townhouse","Loft","Condominium"]

def clean_listings(df,year):
    
    df = df[[x for x in df.columns if x in column_list]]
    df["property_type"] = df["property_type"].str.replace("Entire ","")
    df["property_type"] = df["property_type"].str.replace("Private room in ","")
    df["property_type"] = df["property_type"].str.replace("Shared room in ","")
    df["property_type"] = df["property_type"].str.replace("Room in ","")
    df["property_type"] = df["property_type"].str.title()
    df = df[df["property_type"].isin(property_list)]
    df["database_year"] = year
    
    Bronx, Brooklyn, Manhattan, Queens, StatenIsland = [x for _, x in df.groupby(df["neighbourhood_group_cleansed"])]
    df_list = (Manhattan , Brooklyn , Queens , Bronx , StatenIsland)
       
    listings_Bronx.append(Bronx)
    listings_Brooklyn.append(Brooklyn)
    listings_Manhattan.append(Manhattan)
    listings_Queens.append(Queens)
    listings_StatenIsland.append(StatenIsland)

    return df

def get_amenities(df):
    
    df_clusterid = df[["id","cluster"]]
    df_amenities = df[["amenities"]]
    
    df_amenities["amenities"] = df_amenities["amenities"].str.replace("{", "[").replace("}","]")
    df_amenities = df_amenities["amenities"].str.replace('\[|"|\]',"").str.get_dummies(",")
    df_amenities.columns = df_amenities.columns.str.replace(" ","").str.replace("}","")
    df_amenities.columns = df_amenities.columns.str.split("\\").str[0]
    df_amenities = df_amenities.groupby(level=0, axis=1).sum()
    
    df_amenities[["id","cluster"]] = df_clusterid[["id","cluster"]]
    df = df.drop(["amenities"], axis = 1)
    

    return  df_amenities, df

#Open all listings datasets 
df20 = pd.read_csv("Raw_Datasets/listings2020.csv")
df19 = pd.read_csv("Raw_Datasets/listings2019.csv")
df18 = pd.read_csv("Raw_Datasets/listings2018.csv")
df17 = pd.read_csv("Raw_Datasets/listings2017.csv")
   
#Cleaning all the listings
clean_listings(df20,2020)
clean_listings(df19,2019)
clean_listings(df18,2018)
clean_listings(df17,2017)

#Creating df for each borough and one for all
Bronx = pd.concat(listings_Bronx)
Brooklyn = pd.concat(listings_Brooklyn)
Manhattan = pd.concat(listings_Manhattan)
Queens = pd.concat(listings_Queens)
StatenIsland = pd.concat(listings_StatenIsland)

#Creating a list to clustering each borough
borough_list = [Bronx, Brooklyn, Manhattan, Queens, StatenIsland]

for df in borough_list:
    
    #Getting the geopoints of each listing
    geopoints = df[["latitude","longitude"]].to_numpy()    
    
    #Clustering by Kmeans
    kmeans = KMeans(n_clusters=10)
    kmeans.fit(geopoints)
    clusters = kmeans.fit_predict(geopoints)
    df["cluster"] = clusters    

#Getting the amenities dataset by borough 
Bronx_amenities, Bronx_listings = get_amenities(Bronx)
Brooklyn_amenities, Brooklyn_listings = get_amenities(Brooklyn)
Manhattan_amenities, Manhattan_listings = get_amenities(Manhattan)
Queens_amenities, Queens_listings = get_amenities(Queens)
StatenIsland_amenities, StatenIsland_listings = get_amenities(StatenIsland)
    
#Creating a listings with all boroughs
Listings = pd.concat([Bronx_listings, Brooklyn_listings, Manhattan_listings, Queens_listings, StatenIsland_listings])

#Creating a CSV for each borough listings df
Listings.to_csv("All_listings.csv", sep ="," , index = False)
Bronx_listings.to_csv("Bronx_listings.csv", sep ="," , index = False)
Brooklyn_listings.to_csv("Brooklyn_listings.csv", sep ="," , index = False)
Manhattan_listings.to_csv("Manhattan_listings.csv", sep ="," , index = False)
Queens_listings.to_csv("Queens_listings.csv", sep ="," , index = False)
StatenIsland_listings.to_csv("StatenIsland_listings.csv", sep ="," , index = False)

#Creating a CSV for each borough amenities df
Bronx_amenities.to_csv("Bronx_amenities.csv", sep ="," , index = False)
Brooklyn_amenities.to_csv("Brooklyn_amenities.csv", sep ="," , index = False)
Manhattan_amenities.to_csv("Manhattan_amenities.csv", sep ="," , index = False)
Queens_amenities.to_csv("Queens_amenities.csv", sep ="," , index = False)
StatenIsland_amenities.to_csv("StatenIsland_amenities.csv", sep ="," , index = False)

# Cleaning and merging all calendar datasets

In [17]:
def clean_calendar(CSV):

    df = pd.read_csv(CSV)
    df = df.rename(columns={"listing_id" : "id"})
    df = df[["id","date","price"]]

    df = df[df["price"].notnull()]
    df["price"] = df["price"].str.split(".").str[0]
    df["price"] = df["price"].str.replace(",","")
    df["price"] = df["price"].str.replace("$","")
    df["price"] = df["price"].astype(int)
    
    df["date"] = pd.to_datetime(df["date"])
    df["year"] = df["date"].dt.year
    df["month"] = df["date"].dt.month
    df = df.drop(["date"], axis = 1)
    
    df = df.groupby(["id","year","month"]).describe().iloc[:,1].reset_index()
    df.columns = ["id","year","month","price"]

    return df

cal_Bronx = []
cal_Brooklyn = []
cal_Manhattan = []
cal_Queens = []
cal_StatenIsland = []

Bronx_id = Bronx.id.to_list()
Brooklyn_id = Brooklyn.id.to_list()
Manhattan_id = Manhattan.id.to_list()
Queens_id = Queens.id.to_list()
StatenIsland_id = StatenIsland.id.to_list()

def borough_calendar(df):
    
    df1 = df[df["id"].isin(Bronx_id)]
    df2 = df[df["id"].isin(Brooklyn_id)]
    df3 = df[df["id"].isin(Manhattan_id)]
    df4 = df[df["id"].isin(Queens_id)]
    df5 = df[df["id"].isin(StatenIsland_id)]
        
    cal_Bronx.append(df1)
    cal_Brooklyn.append(df2)
    cal_Manhattan.append(df3)
    cal_Queens.append(df4)
    cal_StatenIsland.append(df5)
    
    return df1, df2, df3, df4, df5

def concat_calendars(cal_list):
    
    df = pd.concat(cal_list)
    df = df.groupby(["id","year","month"]).mean().reset_index()
    df = df[df["year"]>2017]
    df["price"] = df["price"].astype(int)
    
    return df

#Cleaning the calendar datasets
calendar2021 = clean_calendar("Raw_Datasets/calendar2021.csv")
calendar2020 = clean_calendar("Raw_Datasets/calendar2020.csv")
calendar2019 = clean_calendar("Raw_Datasets/calendar2019.csv")
calendar2018 = clean_calendar("Raw_Datasets/calendar2018.csv")
calendar2017 = clean_calendar("Raw_Datasets/calendar2017.csv")

#Spliting by borough
borough_calendar(calendar2021)
borough_calendar(calendar2020)
borough_calendar(calendar2019)
borough_calendar(calendar2018)
borough_calendar(calendar2017)

#Concatenating by borough
Bronx_calendar = concat_calendars(cal_Bronx)
Brooklyn_calendar = concat_calendars(cal_Brooklyn)
Manhattan_calendar = concat_calendars(cal_Manhattan)
Queens_calendar = concat_calendars(cal_Queens)
StatenIsland_calendar = concat_calendars(cal_StatenIsland)

#Creating CSV file for each borough
Bronx_calendar.to_csv("Bronx_calendar.csv", sep ="," , index = False)
Brooklyn_calendar.to_csv("Brooklyn_calendar.csv", sep ="," , index = False)
Manhattan_calendar.to_csv("Manhattan_calendar.csv", sep ="," , index = False)
Queens_calendar.to_csv("Queens_calendar.csv", sep ="," , index = False)
StatenIsland_calendar.to_csv("StatenIsland.csv", sep ="," , index = False)